In [34]:
# Use with NHL_Model_Data_Transform_v5.py
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt

from tensorflow import keras


rng = 69

In [35]:
df = pd.read_csv(r"C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Transformed.csv")
#df = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Transformed_PCA - Test v1.csv")

In [36]:
df = df.drop(columns=['team_Home','team_Away','gameId','home_or_away_Home','home_or_away_Away','win_or_lose_Away'])

In [37]:
# df.hist(bins=50, figsize=(20,15))
# plt.show()

In [38]:
# Heatmap to visualize correlations
corr_matrix = df.corr(numeric_only=True)
# sns.heatmap(corr_matrix, cmap="PiYG")

In [39]:
corr_matrix['win_or_lose_Home'].sort_values(ascending=False)

win_or_lose_Home             1.000000
seasonPointsPerGame_Home     0.301171
fenwickPercentageAvg_Home    0.158833
goalDiffAvg_Home             0.111469
pointsFromGameAvg_Home       0.105522
hitsDiffAvg_Away             0.074194
reboundsForAvg_Home          0.059812
penaltiesAgainstTotal        0.028012
penaltiesForTotal           -0.019402
hitsDiffAvg_Home            -0.038123
reboundsForAvg_Away         -0.057037
pointsFromGameAvg_Away      -0.112578
goalDiffAvg_Away            -0.113146
fenwickPercentageAvg_Away   -0.164881
seasonPointsPerGame_Away    -0.292426
Name: win_or_lose_Home, dtype: float64

In [40]:
# Visualize distributions of attributes
# from pandas.plotting import scatter_matrix

# attributes = ['fenwickPercentageAvg_Home','goalDiffAvg_Home','seasonPointsPerGame_Home','seasonPointsPerGame_Away']
# scatter_matrix(df[attributes], figsize=(12,8));

In [41]:
# These are tentative just to test
df_Final = df
# df_Final = df.drop(columns=['corsiPercentageAvg_Away','penaltiesAgainstTotal','shotsOnGoalDiffAvg_Away'])

In [42]:
from sklearn.model_selection import train_test_split

# Split into 80% training data and 20% testing data
train_set, test_set = train_test_split(df_Final, test_size=0.2, random_state=rng)

nhl = train_set.drop('win_or_lose_Home', axis=1)
nhl_labels = train_set['win_or_lose_Home'].copy()

X_train = train_set.drop('win_or_lose_Home', axis=1)
Y_train = train_set['win_or_lose_Home'].copy()

X_valid = train_set.drop('win_or_lose_Home', axis=1)
Y_valid = train_set['win_or_lose_Home'].copy()

X_test = test_set.drop('win_or_lose_Home', axis=1)
Y_test = test_set['win_or_lose_Home'].copy()

df_Final = df_Final.drop('win_or_lose_Home', axis=1)

In [43]:
# Pipeline constructor used to run transformation steps in order
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

X_train_prepared= num_pipeline.fit_transform(X_train)
X_valid_prepared= num_pipeline.fit_transform(X_valid)
X_test_prepared= num_pipeline.fit_transform(X_test)


In [44]:
# Fine Tune the hyperparameters of RandomForest model
# from sklearn.model_selection import GridSearchCV

# param_grid = [{'n_estimators' : [100,500,1000], 'max_leaf_nodes': [10,16,25,50], 'n_jobs' : [-1]},
#               {'bootstrap': [False], 'n_estimators' : [100,500,1000], 'max_leaf_nodes': [10,16,25,50], 'n_jobs' : [-1]},]

# rnd_clf_test = RandomForestClassifier()

# grid_search = GridSearchCV(rnd_clf_test, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

# grid_search.fit(nhl_prepared,nhl_labels)

# grid_search.best_params_

In [45]:
# cvres = grid_search.cv_results_
# results = []
# for mean_scores, params in zip(cvres["mean_test_score"], cvres["params"]):
#     results.append([np.sqrt(-mean_scores),params])

# results2 = sorted(results, key=lambda x: x[0])
# print(*results2, sep = '\n')

In [46]:
# # baseline model
# from tensorflow import keras


# # create model
# base_model = keras.models.Sequential([
#     keras.Input(shape=(X_train.shape[1],)),
#     keras.layers.Dense(30, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
#                        ])
    
# # Compile model
# base_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [114]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[X_train.shape[1]]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons,activation='selu',kernel_initializer="lecun_normal"))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy',optimizer=optimizer)
    return model

In [113]:
from scikeras.wrappers import KerasClassifier

# Seems like you need to specify some items again when using the scikeras wrapper
keras_clf = KerasClassifier(model=build_model, n_hidden=1, n_neurons=30, learning_rate=3e-3)


In [115]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    'model__n_hidden': [0,1,2],
    'model__n_neurons': np.arange(1,30),
    'model__learning_rate' : reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_clf, param_distribs, n_iter=5, cv=3, error_score='raise')

early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
rnd_search_cv.fit(X_train_prepared,Y_train, epochs=30,
                 validation_data = (X_valid_prepared, Y_valid),
                 callbacks=[early_stopping_cb])

Epoch 1/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6583 - val_loss: 0.5998
Epoch 2/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6081 - val_loss: 0.5972
Epoch 3/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6108 - val_loss: 0.6002
Epoch 4/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6054 - val_loss: 0.6018
Epoch 5/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5931 - val_loss: 0.5972
Epoch 6/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6006 - val_loss: 0.5889
Epoch 7/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5856 - val_loss: 0.6001
Epoch 8/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5972 - val_loss: 0.5885
Epoch 9/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5686 - val_loss: 0.5869
Epoch 10/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5723 - val_loss: 0.5840
Epoch 11/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5864 - val_loss: 0.5934
Epoch 12/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5844 - val_lo

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5808 - val_loss: 0.5922
Epoch 9/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5849 - val_loss: 0.6005
Epoch 10/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5952 - val_loss: 0.5916
Epoch 11/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5844 - val_loss: 0.5851
Epoch 12/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5746 - val_loss: 0.5985
Epoch 13/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5913 - val_loss: 0.5850
Epoch 14/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5612 - val_loss: 0.5830
Epoch 15/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5854 - val_loss: 0.5895
Epoch 16/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5463 - val_loss: 0.5889
Epoch 17/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5781 - val_loss: 0.5868
Epoch 18/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5562 - val_loss: 0.5839
Epoch 19/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5737 - val_loss: 

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5980 - val_loss: 0.5963
Epoch 16/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5880 - val_loss: 0.5955
Epoch 17/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5988 - val_loss: 0.5947
Epoch 18/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6000 - val_loss: 0.5941
Epoch 19/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6015 - val_loss: 0.5936
Epoch 20/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5879 - val_loss: 0.5931
Epoch 21/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5990 - val_loss: 0.5927
Epoch 22/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5966 - val_loss: 0.5922
Epoch 23/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5903 - val_loss: 0.5919
Epoch 24/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5856 - val_loss: 0.5916
Epoch 25/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5843 - val_loss: 0.5913
Epoch 26/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6004 - val_loss:

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5982 - val_loss: 0.5956
Epoch 23/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5781 - val_loss: 0.5882
Epoch 24/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5991 - val_loss: 0.5970
Epoch 25/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5820 - val_loss: 0.5878
Epoch 26/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5730 - val_loss: 0.5994
Epoch 27/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5988 - val_loss: 0.5911
Epoch 28/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5827 - val_loss: 0.6208
Epoch 29/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5884 - val_loss: 0.6027
Epoch 30/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5980 - val_loss: 0.5962
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step
Epoch 1/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6815 - val_loss: 0.6011
Epoch 2/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5982 - val_loss: 0.6002
Epoch 3/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5828 - val_loss: 0.6088
Epoch 30/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5954 - val_loss: 0.6086
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step
Epoch 1/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6630 - val_loss: 0.5994
Epoch 2/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6004 - val_loss: 0.5968
Epoch 3/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6184 - val_loss: 0.5981
Epoch 4/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5907 - val_loss: 0.6003
Epoch 5/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6085 - val_loss: 0.5940
Epoch 6/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6007 - val_loss: 0.5951
Epoch 7/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5995 - val_loss: 0.5972
Epoch 8/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5933 - val_loss: 0.5940
Epoch 9/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6005 - val_loss: 0.5959
Epoch 10/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5760 - val_loss: 0.5974
Epoch 7/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5960 - val_loss: 0.5936
Epoch 8/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5884 - val_loss: 0.5961
Epoch 9/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5774 - val_loss: 0.5973
Epoch 10/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5965 - val_loss: 0.5936
Epoch 11/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5886 - val_loss: 0.5942
Epoch 12/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5758 - val_loss: 0.5938
Epoch 13/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5832 - val_loss: 0.5942
Epoch 14/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5856 - val_loss: 0.5951
Epoch 15/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5671 - val_loss: 0.5931
Epoch 16/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5978 - val_loss: 0.5929
Epoch 17/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5868 - val_loss: 0.

Epoch 14/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6276 - val_loss: 0.6225
Epoch 15/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6297 - val_loss: 0.6187
Epoch 16/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6249 - val_loss: 0.6158
Epoch 17/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6089 - val_loss: 0.6129
Epoch 18/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6098 - val_loss: 0.6104
Epoch 19/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6078 - val_loss: 0.6085
Epoch 20/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6034 - val_loss: 0.6065
Epoch 21/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6125 - val_loss: 0.6050
Epoch 22/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6100 - val_loss: 0.6036
Epoch 23/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6098 - val_loss: 0.6024
Epoch 24/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6028 - val_loss: 0.6015
Epoch 25/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5944

102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5946 - val_loss: 0.5931
Epoch 22/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5849 - val_loss: 0.5929
Epoch 23/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6046 - val_loss: 0.5927
Epoch 24/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5877 - val_loss: 0.5926
Epoch 25/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5876 - val_loss: 0.5925
Epoch 26/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6039 - val_loss: 0.5924
Epoch 27/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5776 - val_loss: 0.5923
Epoch 28/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5850 - val_loss: 0.5922
Epoch 29/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5942 - val_loss: 0.5921
Epoch 30/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5871 - val_loss: 0.5921


RandomizedSearchCV(cv=3, error_score='raise',
                   estimator=KerasClassifier(learning_rate=0.003, model=<function build_model at 0x000001485600A670>, n_hidden=1, n_neurons=30),
                   n_iter=5,
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000148665C9130>,
                                        'model__n_hidden': [0, 1, 2],
                                        'model__n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])})

In [108]:
#print(rnd_search_cv.best_params_)
#print(rnd_search_cv.best_score_)

{'model__learning_rate': 0.0034575031724277975, 'model__n_hidden': 0, 'model__n_neurons': 13}
0.6773897058823529


In [116]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)

{'model__learning_rate': 0.0015024136495751168, 'model__n_hidden': 0, 'model__n_neurons': 2}
0.6767769607843137


In [98]:
model = rnd_search_cv.best_estimator_.model

In [99]:
from scikeras.wrappers import KerasClassifier

# Seems like you need to specify some items again when using the scikeras wrapper
keras_clf = KerasClassifier(model=model, epochs=50, batch_size=5, verbose=0, random_state = 42,
                           loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

# Stop early if model is not getting better after # of patience epochs, restore to best model
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
#checkpoint_cb = keras.callbacks.ModelCheckpoint("Regression_Sequential.keras", save_best_only = True)

keras_clf.fit(X_train_prepared, Y_train, validation_data = (X_valid_prepared, Y_valid),
             callbacks=[early_stopping_cb])



KerasClassifier(
	model=<function build_model at 0x0000014821507550>
	build_fn=None
	warm_start=False
	random_state=42
	optimizer=adam
	loss=binary_crossentropy
	metrics=accuracy
	batch_size=5
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=50
	class_weight=None
)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

#kfold = StratifiedKFold(n_splits=5, shuffle=True)
#results = cross_val_score(keras_clf, X_train_prepared, Y_train, cv=kfold)

#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [100]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
Y_pred = cross_val_predict(keras_clf, X_train_prepared, Y_train, cv=kfold)

from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train, Y_pred)
# [Predict Loss - Actual Loss  |  Predict Win - Actual Loss]
# [Predict Loss - Actual Win   |  Predict Win - Actual Win]

array([[ 966,  561],
       [ 575, 1162]], dtype=int64)

In [101]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

print('When claiming is a win, correct % is ', precision_score(Y_train, Y_pred))
print('Detected % of wins' ,recall_score(Y_train, Y_pred))
print('F1 score is ', f1_score(Y_train, Y_pred))
print('Training Accuracy score is ', accuracy_score(Y_train, Y_pred))


When claiming is a win, correct % is  0.6744051073708648
Detected % of wins 0.6689694876223373
F1 score is  0.6716763005780347
Training Accuracy score is  0.6519607843137255


In [102]:
# Train Accuracy
Y_train_pred = keras_clf.predict(X_train_prepared)
print(accuracy_score(Y_train, Y_train_pred))

Y_test_pred = keras_clf.predict(X_test_prepared)
print(accuracy_score(Y_test, Y_test_pred))

0.7208946078431373
0.6629901960784313


In [ ]:
df_Predict = pd.read_csv(r"C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Predict.csv")
#df_Predict = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Predict.csv")

In [ ]:
X_real_predict = df_Predict.iloc[:,2:]

np.array((df_Predict.iloc[:,1].values,keras_clf.predict(X_real_predict))).T